In [12]:
import pandas as pd
import tensorflow as tf

c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\deves\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\deves\appd

In [4]:
diabetes = pd.read_csv('pima-indians-diabetes.csv')

In [7]:
diabetes.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [8]:
diabetes.columns

Index(['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree', 'Age', 'Class', 'Group'],
      dtype='object')

In [9]:
cols_to_norm = ['Number_pregnant', 'Glucose_concentration', 'Blood_pressure', 'Triceps',
       'Insulin', 'BMI', 'Pedigree']

In [10]:
diabetes[cols_to_norm] = diabetes[cols_to_norm].apply(lambda x: (x - x.min())/(x.max() - x.min()))

In [11]:
diabetes.head()

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [15]:
num_preg = tf.feature_column.numeric_column("Number_pregnant")
gluc_conc = tf.feature_column.numeric_column("Glucose_concentration")
bp = tf.feature_column.numeric_column("Blood_pressure")
tric = tf.feature_column.numeric_column("Triceps")
insulin = tf.feature_column.numeric_column("Insulin")
bmi = tf.feature_column.numeric_column("BMI")
pedi = tf.feature_column.numeric_column("Pedigree")
age = tf.feature_column.numeric_column("Age")

In [16]:
assigned_group = tf.feature_column.categorical_column_with_vocabulary_list("Group",vocabulary_list=["A","B","C","D"])

In [22]:
feature_columns = [num_preg,gluc_conc,bp,tric,insulin,bmi,pedi,age,assigned_group]

In [77]:
# x_data = diabetes.drop(["Class"],axis=1)
x_data = diabetes[["Triceps"]]

In [78]:
x_data.head()

,Triceps
0,0.353535
1,0.292929
2,0.000000
3,0.232323
4,0.353535


In [79]:
labels = diabetes["Class"]

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
x_train, x_test, y_train, y_test = train_test_split(x_data,labels,test_size=0.3)

In [82]:
input_fn = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,
                                              batch_size=10,num_epochs=1000,
                                              shuffle=True)

In [83]:
model = tf.estimator.LinearClassifier(feature_columns=feature_columns
                                     ,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\deves\\AppData\\Local\\Temp\\tmp8hm2ghxe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A2D6AC9B70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [84]:
model.train(input_fn=input_fn,steps=1000)

INFO:tensorflow:Calling model_fn.


ValueError: Feature Age is not in features dictionary.

In [53]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,
                                                     batch_size=10,num_epochs=1,
                                                     shuffle=False)

In [54]:
results = model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-08T11:29:08Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\deves\AppData\Local\Temp\tmpuisy39b8\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2022-07-08-11:29:09
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7099567, accuracy_baseline = 0.6450217, auc = 0.7777051, auc_precision_recall = 0.61514753, average_loss = 0.56298876, global_step = 1000, label/mean = 0.35497835, loss = 5.4187665, precision = 0.6744186, prediction/mean = 0.37000042, recall = 0.35365853
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\deves\AppData\Local\Temp\tmpuisy39b8\model.ckpt-1000


In [55]:
results

{'accuracy': 0.7099567,
 'accuracy_baseline': 0.6450217,
 'auc': 0.7777051,
 'auc_precision_recall': 0.61514753,
 'average_loss': 0.56298876,
 'label/mean': 0.35497835,
 'loss': 5.4187665,
 'precision': 0.6744186,
 'prediction/mean': 0.37000042,
 'recall': 0.35365853,
 'global_step': 1000}

In [56]:
 dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],
                                       feature_columns=feature_columns,
                                       n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\deves\\AppData\\Local\\Temp\\tmph41sx871', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A2D47F2A58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [57]:
embedded_groups = tf.feature_column.embedding_column(assigned_group,
                                                     dimension=4)

In [58]:
feature_columns = [num_preg,gluc_conc,bp,tric,insulin,bmi,pedi,age,embedded_groups]

In [59]:
input_fn_dnn = tf.estimator.inputs.pandas_input_fn(x_train,y_train,
                                                  batch_size=10,
                                                  num_epochs=1000,
                                                  shuffle=True)

In [72]:
dnn_model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10,20,20,20,10,10],n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\deves\\AppData\\Local\\Temp\\tmp4kprhtpu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A2D6B29940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [73]:
dnn_model.train(input_fn=input_fn_dnn,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\deves\AppData\Local\Temp\tmp4kprhtpu\model.ckpt.
INFO:tensorflow:loss = 7.274955, step = 1
INFO:tensorflow:global_step/sec: 364.389
INFO:tensorflow:loss = 6.828619, step = 101 (0.275 sec)
INFO:tensorflow:global_step/sec: 759.601
INFO:tensorflow:loss = 6.842458, step = 201 (0.132 sec)
INFO:tensorflow:global_step/sec: 661.89
INFO:tensorflow:loss = 6.0605683, step = 301 (0.152 sec)
INFO:tensorflow:global_step/sec: 642.733
INFO:tensorflow:loss = 7.624502, step = 401 (0.154 sec)
INFO:tensorflow:global_step/sec: 807.919
INFO:tensorflow:loss = 5.5329485, step = 501 (0.125 sec)
INFO:tensorflow:global_step/sec: 753.883
INFO:tensorflow:loss = 6.239246, step = 601 (0.134 sec)
INFO:tensorflow:global_step/sec: 7

In [74]:
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,
                                                   batch_size=10,
                                                   num_epochs=1,
                                                   shuffle=False)

In [75]:
results = dnn_model.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-08T11:40:14Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\deves\AppData\Local\Temp\tmp4kprhtpu\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2022-07-08-11:40:15
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.64935064, accuracy_baseline = 0.6450217, auc = 0.6995008, auc_precision_recall = 0.44790524, average_loss = 0.5842327, global_step = 1000, label/mean = 0.35497835, loss = 5.6232395, precision = 0.50617284, prediction/mean = 0.38723293, recall = 0.5
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\deves\AppData\Local\Temp\tmp4kprhtpu\model.ckpt-1000


In [76]:
results

{'accuracy': 0.64935064,
 'accuracy_baseline': 0.6450217,
 'auc': 0.6995008,
 'auc_precision_recall': 0.44790524,
 'average_loss': 0.5842327,
 'label/mean': 0.35497835,
 'loss': 5.6232395,
 'precision': 0.50617284,
 'prediction/mean': 0.38723293,
 'recall': 0.5,
 'global_step': 1000}